In [5]:
# https://www.bfs.admin.ch/bfs/fr/home/services/recherche/api.html
# https://dam-api.bfs.admin.ch/hub/swagger-ui/index.html
# communes
# https://www.bfs.admin.ch/bfs/fr/home/services/recherche/api/api-gemeinde.assetdetail.15224055.html

# motorisation
# https://www.pxweb.bfs.admin.ch/sq/ccc96ddd-61a9-49ab-ad7f-dabbc934e8c0

import requests

# Population
dataset_url = 'https://www.pxweb.bfs.admin.ch/api/v1/en/px-x-0102030000_101/px-x-0102030000_101.px'
dataset_url = 'https://www.pxweb.bfs.admin.ch/api/v1/fr/px-x-1103020100_165/px-x-1103020100_165.px'

metadata_response = requests.get(dataset_url)
metadata_json = metadata_response.json()

response_canton = requests.post(dataset_url, json={
  "query": [
    {
      "code": "Kanton",
      "selection": {
        "filter": "item",
        "values": [
          "1",
          "2",
          "3",
          "4",
          "5",
          "6",
          "7",
          "8",
          "9",
          "10",
          "11",
          "12",
          "13",
          "14",
          "15",
          "16",
          "17",
          "18",
          "19",
          "20",
          "21",
          "22",
          "23",
          "24",
          "25",
          "26",
          "27"
        ]
      }
    },
    {
      "code": "Treibstoff",
      "selection": {
        "filter": "item",
        "values": [
          "100",
          "200",
          "300",
          "400",
          "500",
          "600",
          "9900"
        ]
      }
    }
  ],
  "response": {
    "format": "json-stat"
  }
})
json = response_canton.json()

In [10]:
import pandas as pd

values = {"values": json['dataset']['value']}
length = 1
total_length = len(json['dataset']['value'])
for dimension_id in json['dataset']['dimension']['id']:
    dimension = json['dataset']['dimension'][dimension_id]
    
    current_length = len(dimension['category']['index'])
    nb = int(total_length / (length * current_length))
    
    v = list(json['dataset']['value'])
    for x in range(length):
        for y, value in enumerate(dimension['category']['label'].values()):
            for z in range(nb):
                v[x * current_length * nb + y * nb + z] = value
    
    values[dimension['label']] = v
            
    length *= current_length

df = pd.DataFrame(values)
#print(json['dataset']['value'])
df

,values,Canton,Carburant,Année
0,86124,Zürich,Essence,2005
1,87905,Zürich,Essence,2006
2,90450,Zürich,Essence,2007
3,93180,Zürich,Essence,2008
4,93780,Zürich,Essence,2009
...,...,...,...,...
3208,0,Confédération,Autre,2017
3209,0,Confédération,Autre,2018
3210,0,Confédération,Autre,2019
3211,0,Confédération,Autre,2020


In [28]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output


# Build App
app = JupyterDash('Carburant')
app.layout = html.Div([
    html.H1("Carburant"),
    dcc.Graph(id='graph'),
    html.Label([
        "Canton",
        dcc.Dropdown(
            id='Canton-dropdown', clearable=False,
            value='plasma', options=[
                {'label': c, 'value': c}
                for c in set(df['Canton'])
            ])
    ]),
])

# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    [Input("Canton-dropdown", "value")]
)
def update_figure(canton):
    fig = px.line(df.query(f'Canton == "{canton}"'), x="Année", y="values", color='Carburant')
    fig.update_layout(yaxis_type="log",
        xaxis = {
            'rangeslider': {
                'visible': True
            },
        }
    )
    return fig

# Run app and display result inline in the notebook
app.run_server(mode='inline')